# 기초상품 추천 코드 구현해보기
주어진 데이터를 활용하여 유저 아이디와 원하는 제품 유형을 입력하면   
해당 유저와 유사도가 높은 유저의 평점 높은 제품을 추천해주는 함수 만들기

* 유저데이터 파일명 : user_data
* 제품데이터 파일명 : product_data

In [1]:
import pandas as pd

#유저데이터 불러오기
u_data = pd.read_csv("user_data.csv")

#제품데이터 불러오기
p_data = pd.read_csv("product_data.csv")

In [2]:
#product_id 기준으로 유저,제품 데이터 합쳐서 df에 저장하고 user_id를 기준으로 오름차순 정렬

df=pd.merge(p_data, u_data, on='product_id').sort_values(by='user_id')
df.head()

,product_id,product_name,product_type,target_skin_type,ingredient,Unnamed: 5,Unnamed: 6,user_id,rating,skin
895,24.0,serum_3,세럼,민감성,"쑥잎추출물, 정제수, 글리세린, 다이프로필렌글라이콜, 판테놀, 1,2-헥산다이올, ...",NaN,NaN,1,4,민감성
1440,38.0,toner_6,토너,건성,"정제수, 다이프로필렌글라이콜, 글리세레스-26, 1,2-헥산다이올, 나이아신아마이드...",NaN,NaN,1,5,민감성
1524,40.0,toner_8,토너,민감성,"정제수, 어성초추출물(=약모밀추출물, 20%), 메틸프로판다이올, 펜틸렌글라이콜, ...",NaN,NaN,1,5,민감성
1288,34.0,lotion_6,로션,건성,"아벤느온천수(아쿠아),글리세린,카프릴릭/카프릭트리글리세라이드,이소프로필팔미테이트,프...",NaN,NaN,1,4,민감성
1249,33.0,serum_6,세럼,지성,"아쿠아(아벤느 온천수),아이소프로필알코올,피이지-6,글리세린,흰무늬엉겅퀴열매추출물,...",NaN,NaN,1,1,민감성


In [3]:
#각 유저가 제품에 남긴 평점들로 이루어진 데이터 프레임(df_users) 만들기 인덱스:user_id, 컬럼명:product_id
df_users=df.pivot_table('rating', index ='user_id', columns='product_id')
df_users.fillna(0, inplace=True)
df_users.head()

product_id,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,51.0,52.0,53.0,54.0,55.0,56.0,57.0,58.0,59.0,60.0
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,2.0,0.0,5.0,0.0,2.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
2,2.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,2.0,0.0,0.0,4.0,3.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
#유저간 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity
cos_matrix = cosine_similarity(df_users.values, df_users.values)
cos_matrix


array([[1.        , 0.37756255, 0.11836501, ..., 0.30848211, 0.30876888,
        0.33248992],
       [0.37756255, 1.        , 0.10316111, ..., 0.22016375, 0.30076748,
        0.11435019],
       [0.11836501, 0.10316111, 1.        , ..., 0.3901178 , 0.2384219 ,
        0.27498715],
       ...,
       [0.30848211, 0.22016375, 0.3901178 , ..., 1.        , 0.09955444,
        0.40333349],
       [0.30876888, 0.30076748, 0.2384219 , ..., 0.09955444, 1.        ,
        0.23491702],
       [0.33248992, 0.11435019, 0.27498715, ..., 0.40333349, 0.23491702,
        1.        ]])

In [60]:
#위에서 계산한 cos_matrix를 데이터프레임으로 만듭니다. 
df_users_cosine= pd.DataFrame(cos_matrix, index=df_users.index, columns=df_users.index)
df_users_cosine.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.377563,0.118365,0.195347,0.465273,0.350255,0.369234,0.382708,0.209271,0.388365,...,0.434633,0.327872,0.595550,0.253139,0.292668,0.306267,0.256333,0.308482,0.308769,0.332490
2,0.377563,1.000000,0.103161,0.106745,0.226767,0.327382,0.163269,0.323855,0.210206,0.190810,...,0.244048,0.212365,0.221078,0.246255,0.231953,0.218187,0.195578,0.220164,0.300767,0.114350
3,0.118365,0.103161,1.000000,0.328517,0.265719,0.423921,0.313412,0.399815,0.202431,0.194410,...,0.362167,0.376298,0.201629,0.211564,0.175430,0.439941,0.336052,0.390118,0.238422,0.274987
4,0.195347,0.106745,0.328517,1.000000,0.306010,0.280929,0.385025,0.287416,0.299863,0.477361,...,0.165149,0.281387,0.224007,0.263200,0.185619,0.296223,0.209036,0.376882,0.164470,0.357317
5,0.465273,0.226767,0.265719,0.306010,1.000000,0.209787,0.217964,0.221134,0.097609,0.239447,...,0.267159,0.319854,0.243056,0.183303,0.338358,0.361589,0.202548,0.051116,0.259230,0.302407


## 입력한 아이디의 유저와 비슷한 유저가 만족한 제품 추천하는 함수 만들기
1. df_users_cosine에서 입력한 아이디와 유사도 높은 5명을 sim_users로 지정
2. df에서 user_id가 sim_users의 인덱스와 일치하는 값들을 sim_user_df에 할당
3. sim_user_df에서 입력한 product_type과 일치하는 값들을 지정하고 rating을 기준으로 내림차순한 결과를 products에 할당
4. products에서 rating 4점 이상인 값들만 다시 products에 할당
5. products에서 product_name이 중복인 것은 첫번째 값만 남김
6. products의 product_name을 데이터 프레임 result로 만듦 (컬럼명은 나와 비슷한 사용자가 만족한(입력한 제품타입)제품)
7. 함수 마지막에 print(result) 출력

In [71]:
#반드시 skeleton 코드에 맞춰서 작성하지 않아도 됨. 단, 작성한 코드에 대해 어떤 알고리즘으로 결과가 나오게 했는지 설명할 수 있어야함. 

def user_based_recommend(user_id, product_type):
    cosine_id = df_users_cosine[user_id]
    cosine_id[user_id] = 0
    sim_users =cosine_id.sort_values(ascending=False)[:5].index #1 df_users_cosine에서 입력한 아이디와 유사도 높은 5명을 sim_users로 지정
    sim_user_df =  []
    for i in range(0,5):
        sim_user_df.append(df[df['user_id']==sim_users[i]])
    sim_user_df=pd.concat(sim_user_df) #2 df에서 user_id가 sim_users의 인덱스와 일치하는 값들을 sim_user_df에 할당
    
    #3 product_type과 일치하는 값을 선택하고 rating을 기준으로 내림차순으로 정렬
    prdt = sim_user_df[sim_user_df['product_type'] == product_type]
    products = prdt.sort_values(by='rating', ascending=False)
    
    products = products[products['rating'] >= 4] #4 products에서 rating 4점 이상인 값들만 다시 products에 할당
    products.drop_duplicates(['product_name'], keep='first', inplace=True) #5 products에서 product_name이 중복인 것은 첫번째 값만 남김
   
    result = pd.DataFrame(products['product_name'].values, columns=[f'나와 비슷한 사용자가 만족한 {product_type} 타입 제품명']) #6 products의 product_name을 데이터 프레임 result로 만듦 (컬럼명은 나와 비슷한 사용자가 만족한(입력한 제품타입)제품)

    return print(result) #7 함수 마지막에 print(result) 출력

#함수 호출
user_based_recommend(1, "토너")

  나와 비슷한 유저가 만족한 토너 타입 제품명
0                 toner_11
1                  toner_6
2                  toner_1
3                  toner_2
4                  toner_5
5                  toner_8
